<a href="https://colab.research.google.com/github/inspire007/KaggleDatasets/blob/main/house-prices-advanced-regression-techniques/house_prices_advanced_regression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

data = pd.read_csv('train.csv')

X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1] + list(range(4,16)) + list(range(20,25)) + list(range(26,33)) + [34] + list(range(38,42)) + [52,54,56,57,59,62,63,64,71,72,73,77,78])], remainder='passthrough')
X = ct.fit_transform(X)

from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='mean', missing_values=np.nan)
X = si.fit_transform(X)

#test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=500, random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

#print(y_pred, y_test)

r2 = r2_score(y_test, y_pred)
print("Random forest r2 score: ", r2, "\n\n")

from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(random_state=42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("Decision tree r2 score: ", r2, "\n\n")

from sklearn.svm import SVR

regr = SVR(kernel='rbf')
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("Non-linear SVR r2 score: ", r2, "\n\n")

from sklearn.linear_model import LinearRegression
regr = LinearRegression()

regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("Linear regr r2 score: ", r2, "\n\n")

from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree = 2)
X_train_poly = pf.fit_transform(X_train)
X_test_poly = pf.transform(X_test)

regr = LinearRegression()

regr.fit(X_train_poly, y_train)
y_pred = regr.predict(X_test_poly)

r2 = r2_score(y_test, y_pred)
print("Poly regr r2 score: ", r2, "\n\n")

#choose random forest based on r2 score

data = pd.read_csv('test.csv')
X_test_actual = data.iloc[:, 1:].values
X_test_actual = ct.transform(X_test_actual) #onehotencoding
X_test_actual = si.transform(X_test_actual) #replace missing values

sc = StandardScaler()
X = sc.fit_transform(X)
X_test_actual = sc.transform(X_test_actual)

regr = RandomForestRegressor(n_estimators=500, random_state=42)
regr.fit(X,y)

y_pred_actual = regr.predict(X_test_actual)
pid = data.iloc[:, 0].values
pid = pid.reshape(len(pid), 1)

output = np.concatenate(
    (
        pid,
        y_pred_actual.reshape(len(y_pred_actual), 1)
    ),
    1
)

np.savetxt('housing_output.csv', output, delimiter=',', comments='', header='Id,SalePrice', fmt=['%d', '%.3f'])

Random forest r2 score:  0.89131062606417 


Decision tree r2 score:  0.7483582880980072 


Non-linear SVR r2 score:  -0.024548274419177618 


Linear regr r2 score:  0.443431521148146 


Poly regr r2 score:  0.8349287889501612 


